In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os


import gc
import pickle as pickle

from sklearn.model_selection import KFold

from catboost import CatBoostRegressor
import lightgbm as lgbm

import itertools

import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [14]:
#tail -n +2 train.csv | split -l 150000
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [4]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [5]:
file = np.random.choice(splits)
df = pd.read_csv(f'train/{file}', float_precision='round_trip', header=0)
columns = ['acoustic_data','time_to_failure']
df.columns = columns

In [12]:
TIMESTEPS=1

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

split_chunks = np.array_split(splits,mp.cpu_count())

#def build_segment_f(splits, number_of_groups,test=False, augment=False, scale=True, noise=0.5):

param_test = False
param_augment = False
param_scale = False
param_noise = 0.5


if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS,param_test,param_augment,param_scale,param_noise]) for chunk in split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
200
200
200
200
200
200
200
200
300
300
300
300
300
300
300
300
400
400
400
400
400
400
400
400
500
500
500
500
500
500
500
500


In [15]:
res[0].get()[100]

,mean,max,min,std,abs,q25,q50,q75,sum,uniq,pos,negs,ssum,imax,imin,abs_nrg,abs_sum_chg,autocorr_10,cid_ce,kurtosis,mean_chg,reocurring_pct,r_sigma,ratio_to_length,skewness,strike_below,strike_above,last_loc_max,first_loc_max,last_loc_min,first_loc_min,perc_reocurr_dp,perc_reocurr_all,sum_reoccurr_val,sum_reoccurr_dp,ratio_value_number,peaks,time_to_failure,augmented
0,4.811387,108.0,-76.0,4.790863,184.0,3.0,5.0,7.0,721708.0,129.0,133141.0,11175.0,818076.0,143729.0,29498.0,6915272.0,389228.0,-0.410707,271.678952,22.444657,0.0,0.875969,0.038907,0.00086,0.210289,23.0,27.0,0.9582,0.958193,0.19666,0.196653,0.875969,0.999893,545.0,721173.0,0.00086,453.0,5.3051,False


In [33]:
columns=res[0].get()[0].columns.values

data = pd.DataFrame(columns=columns)


i=0
for r in res:
    for df in r.get():
        data = data.append(df)
        i+=1
        

data.reset_index(drop=True,inplace=True)


In [ ]:

data.sample(10)

In [34]:
combinations2 = list(itertools.combinations(data.columns.values, 2))
combinations3 = list(itertools.combinations(data.columns.values, 3))


In [22]:

for combination in combinations2:
    if 'augmented' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    feature = f'{f1}_mult_{f2}'
    data[feature] = data[f1] * data[f2]
    feature = f'{f1}_plus_{f2}'
    data[feature] = data[f1] + data[f2]
    feature = f'{f1}_div_{f2}'
    #print(feature)
    data[feature] = data[f1] / data[f2]
    data[feature] = pd.to_numeric(data[feature], downcast='float')


In [24]:

for combination in combinations3:
    if 'augmented' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    f3 = combination[2]
    feature = f'{f1}_mult_{f2}_mult_{f3}'
    data[feature] = data[f1] * data[f2] * data[f3]
    feature = f'{f1}_plus_{f2}_plus_{f3}'
    data[feature] = data[f1] + data[f2] + data[f3]
    feature = f'{f1}_div_{f2}_div_{f3}'
    #print(feature)
    data[feature] = data[f1] / data[f2] / data[f3]
    data[feature] = pd.to_numeric(data[feature], downcast='float')
    

In [ ]:
len(combinations)

In [35]:

test_size = 0.1
#train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=test_size, random_state=42)

val_size = int(len(data)*0.1)
non_augmented = data[data['augmented'] == False]
val_indices = np.random.choice(non_augmented.index.values, val_size)
val_data = data[data.index.isin(val_indices)].drop('augmented', axis=1)
train_data = data[~data.index.isin(val_indices)].drop('augmented', axis=1)

train_y = train_data['time_to_failure']
val_y = val_data['time_to_failure']

train_data = train_data.drop('time_to_failure', axis=1)
val_data = val_data.drop('time_to_failure', axis=1)

In [36]:
features = train_data.columns.values

In [41]:

cb_model = CatBoostRegressor(iterations=10000,
                             #learning_rate=0.05,
                             eval_metric='MAE',
                             #task_type = "GPU",
                             use_best_model=True,
                             od_type = "Iter",
                             od_wait = 20,
                             #depth=3,
                             border_count=254,
                             #bagging_temperature = 20,
                             #cat_features=[0],
                             random_seed = 42)



cb_model.fit(train_data[features], train_y, #cat_features=categorical_features_indices,
             eval_set=(val_data[features],val_y),
             #cat_features=categorical_features_pos,         
             verbose=True)

scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

0:	learn: 5.5387513	test: 5.3521091	best: 5.3521091 (0)	total: 70.1ms	remaining: 11m 41s
1:	learn: 5.3840508	test: 5.1983856	best: 5.1983856 (1)	total: 80.7ms	remaining: 6m 43s
2:	learn: 5.2338774	test: 5.0485769	best: 5.0485769 (2)	total: 91.6ms	remaining: 5m 5s
3:	learn: 5.0899611	test: 4.9071396	best: 4.9071396 (3)	total: 102ms	remaining: 4m 15s
4:	learn: 4.9519051	test: 4.7707590	best: 4.7707590 (4)	total: 113ms	remaining: 3m 45s
5:	learn: 4.8207730	test: 4.6435209	best: 4.6435209 (5)	total: 123ms	remaining: 3m 24s
6:	learn: 4.6951293	test: 4.5216865	best: 4.5216865 (6)	total: 132ms	remaining: 3m 7s
7:	learn: 4.5749264	test: 4.4043618	best: 4.4043618 (7)	total: 141ms	remaining: 2m 56s
8:	learn: 4.4582591	test: 4.2924304	best: 4.2924304 (8)	total: 155ms	remaining: 2m 52s
9:	learn: 4.3472953	test: 4.1839236	best: 4.1839236 (9)	total: 166ms	remaining: 2m 45s
10:	learn: 4.2412727	test: 4.0817149	best: 4.0817149 (10)	total: 179ms	remaining: 2m 42s
11:	learn: 4.1396747	test: 3.9845188	be

103:	learn: 2.0871673	test: 2.0288161	best: 2.0273105 (99)	total: 1.11s	remaining: 1m 45s
104:	learn: 2.0865142	test: 2.0293905	best: 2.0273105 (99)	total: 1.12s	remaining: 1m 45s
105:	learn: 2.0861411	test: 2.0293667	best: 2.0273105 (99)	total: 1.13s	remaining: 1m 45s
106:	learn: 2.0857106	test: 2.0299114	best: 2.0273105 (99)	total: 1.14s	remaining: 1m 45s
107:	learn: 2.0852114	test: 2.0296968	best: 2.0273105 (99)	total: 1.16s	remaining: 1m 45s
108:	learn: 2.0852973	test: 2.0298395	best: 2.0273105 (99)	total: 1.17s	remaining: 1m 45s
109:	learn: 2.0846980	test: 2.0302147	best: 2.0273105 (99)	total: 1.18s	remaining: 1m 45s
110:	learn: 2.0843015	test: 2.0305402	best: 2.0273105 (99)	total: 1.19s	remaining: 1m 45s
111:	learn: 2.0840790	test: 2.0304022	best: 2.0273105 (99)	total: 1.19s	remaining: 1m 45s
112:	learn: 2.0831813	test: 2.0296470	best: 2.0273105 (99)	total: 1.2s	remaining: 1m 45s
113:	learn: 2.0833071	test: 2.0300333	best: 2.0273105 (99)	total: 1.21s	remaining: 1m 45s
114:	learn:

[('negs', 22.00019706772714),
 ('abs_sum_chg', 21.307451594328242),
 ('pos', 13.68451466481485),
 ('ssum', 13.60870876511053),
 ('abs_nrg', 6.490830010597606),
 ('r_sigma', 5.500908633667382),
 ('sum_reoccurr_dp', 3.806805425885695),
 ('mean', 3.5122415481141354),
 ('kurtosis', 1.9104347360400766),
 ('strike_above', 0.8097808104072409),
 ('imin', 0.7971303020277785),
 ('max', 0.7835033773241783),
 ('reocurring_pct', 0.7269578792275802),
 ('sum_reoccurr_val', 0.6959273011405624),
 ('peaks', 0.6897052434064554),
 ('min', 0.6191711824085099),
 ('skewness', 0.4779916571627522),
 ('imax', 0.47553533216640087),
 ('mean_chg', 0.46435346087728185),
 ('last_loc_min', 0.4367972302523331),
 ('autocorr_10', 0.41317084836526685),
 ('perc_reocurr_all', 0.4110004254445803),
 ('strike_below', 0.3768825035034483)]

In [39]:
lgtrain = lgbm.Dataset(train_data[features], label=train_y)
lgval = lgbm.Dataset(val_data[features], label=val_y)


# needs to keep overfiting under control, default params don't cut it at all
params = {
        "num_threads": 8,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "mae",
        "seed": 42,
        "learning_rate" : 0.05,
        #"min_data_in_leaf": 100,
        #"num_leaves": 10,
        #"max_depth" : 4,
        #"bagging_fraction": 0.7,
        #"bagging_freq": 1,
        #"feature_fraction": 0.7,
        #"lambda_l1": 10,
}

evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 10000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=30, 
                      verbose_eval=10, 
                      evals_result=evals_result)


scores = {}
for i,score in enumerate(model_lgb.feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

Training until validation scores don't improve for 30 rounds.
[10]	valid_0's l1: 2.45127
[20]	valid_0's l1: 2.22033
[30]	valid_0's l1: 2.12379
[40]	valid_0's l1: 2.09014
[50]	valid_0's l1: 2.06915
[60]	valid_0's l1: 2.06181
[70]	valid_0's l1: 2.04493
[80]	valid_0's l1: 2.03691
[90]	valid_0's l1: 2.02596
[100]	valid_0's l1: 2.02819
[110]	valid_0's l1: 2.03394
[120]	valid_0's l1: 2.03369
Early stopping, best iteration is:
[94]	valid_0's l1: 2.0253


[('negs', 209),
 ('mean', 199),
 ('ssum', 170),
 ('reocurring_pct', 163),
 ('sum_reoccurr_dp', 159),
 ('sum_reoccurr_val', 152),
 ('skewness', 152),
 ('pos', 151),
 ('imax', 130),
 ('autocorr_10', 128),
 ('r_sigma', 125),
 ('max', 120),
 ('strike_below', 112),
 ('peaks', 100),
 ('abs_sum_chg', 100),
 ('abs_nrg', 96),
 ('kurtosis', 91),
 ('last_loc_min', 90),
 ('strike_above', 84),
 ('imin', 83),
 ('min', 72),
 ('perc_reocurr_all', 68),
 ('mean_chg', 66)]

In [38]:
features = [item[0] for item in scores.items() if item[1] > 50]

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor

lr_features = features[0:10]

lr_model =  Lasso(alpha=0.5)
lr_model.fit(train_data[lr_features], train_y)

from sklearn.metrics import mean_absolute_error

train_preds = lr_model.predict(val_data[lr_features])
print("mae: ", mean_absolute_error(val_y, train_preds))

In [ ]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [ ]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())
    
    
param_test = True
param_augment = False
param_scale = False
param_noise = 0.6


if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS,param_test,param_augment,param_scale,param_noise]) for chunk in test_split_chunks]
    pool.close()
    pool.join()

In [ ]:
columns=res[0].get()[0].columns.values

test_data = pd.DataFrame(columns=columns)


i=0
for r in res:
    for df in r.get():
        #print(df)
        test_data = test_data.append(df)
        #print(len(test_data))
        i+=1
        

test_data.reset_index(drop=True,inplace=True)
print(len(test_data))

In [ ]:
combinations2 = list(itertools.combinations(test_data.columns.values, 2))
combinations3 = list(itertools.combinations(test_data.columns.values, 3))


In [ ]:

for combination in combinations2:
    if 'seg_id' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    feature = f'{f1}_mult_{f2}'
    test_data[feature] = test_data[f1] * test_data[f2]
    feature = f'{f1}_plus_{f2}'
    test_data[feature] = test_data[f1] + test_data[f2]
    feature = f'{f1}_div_{f2}'
    #print(feature)
    test_data[feature] = test_data[f1] / test_data[f2]
    test_data[feature] = pd.to_numeric(test_data[feature], downcast='float')


In [ ]:

for combination in combinations3:
    if 'seg_id' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    f3 = combination[2]
    feature = f'{f1}_mult_{f2}_mult_{f3}'
    test_data[feature] = test_data[f1] * test_data[f2] * test_data[f3]
    feature = f'{f1}_plus_{f2}_plus_{f3}'
    test_data[feature] = test_data[f1] + test_data[f2] + test_data[f3]
    feature = f'{f1}_div_{f2}_div_{f3}'
    #print(feature)
    test_data[feature] = test_data[f1] / test_data[f2] / test_data[f3]
    test_data[feature] = pd.to_numeric(test_data[feature], downcast='float')
    

In [ ]:
ids = test_data['seg_id'].apply(lambda id: id.split('.')[0])

In [ ]:
preds = cb_model.predict(test_data[features])


In [ ]:
lg_preds = model_lgb.predict(test_data[features])

In [ ]:
lr_preds = lr_model.predict(test_data[lr_features])

In [ ]:
preds = (lg_preds * 0.5) + (lr_preds * 0.5)


In [ ]:

submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = lg_preds

submission.to_csv('submission.csv', index=False)

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission.head()

In [ ]:
lgbm_preds = pd.read_csv('lgbm.csv')['time_to_failure']
cb_preds = pd.read_csv('cb1549.csv')['time_to_failure']
lstm_preds = pd.read_csv('lstm1627.csv')['time_to_failure']


preds = (cb_preds * 0.4) + (lgbm_preds * 0.5)

preds.clip(0,20,out=preds)

print(np.mean(preds))
print(np.max(preds))

submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)